## Task: Personalized broker to find apartments in New York City
- Agent has access to tools and personal details to find the best apartment for each person.
- Agent can parse listing and go beyond StreetEasy filters, better understand the renter through large amounts of text, images, etc. and proactively reach out to schedule viewings. 
- Reward function is if the person want's to schedule a viewing (determined by LLM as a judge with personal info for the time being)




## Details
- Chat completions API with claude driving
- Tools:
    - Apartment Text search
    - Analyze apartment images
    - Explore neighborhood
    - Map directions
- Outcomes:
    - Recommend apartment(s)
    - Do nothing


## What I would like to improve on
- Better file search: Should gather more data and consider organizing it by neighborhood, provide more context on the schema
- Better search by keywords (a budget of 4000 should include listings less than 4k but now fails because i'im comparing on ints)
- Larger apartment dataset for testing
- Refine tool usage (web search is one of the more powerful tools and i'd expect to see it called more often)

In [5]:
from openai import OpenAI
import json
import os
import sys
from dotenv import load_dotenv
import asyncio
import nest_asyncio

sys.path.append(os.path.join(os.getcwd(), '..'))
from helpers import call_function_async

nest_asyncio.apply()
load_dotenv()
client = OpenAI()
# client = OpenAI(api_key=os.getenv("ANTHROPIC_API_KEY"), base_url="https://api.anthropic.com/v1/")

agent_model = "gpt-4.1" # claude-opus-4-20250514, claude-sonnet-4-20250514

In [22]:
developer = """
You are a real estate agent in New York City.
You are given a list of apartments and you need to find the best one for the user.
Think through what the user is looking for and then use tools to find the best apartments for them.
You have the following tools to help you:
- get_apartment_dataset_schema: get the schema of the apartment dataset and an example.
- get_apartment_values: get all possible values of a list of fields for all available apartments in the city.
- ask_user: ask the user for more information about what they are looking for.
- search_keywords_in_values: Search for keywords in the values of the apartment.
- search_by_field: Search for keywords in a specific field of the apartment.
- web_search: search for information on the web. 

Alternate between calling some tools, and reasoning about the user's profile.

The default filepath for the apartment dataset is "../data/streeteasy.json".
Communicate directly with the user via the ask_user tool.
After consulting all tools return the best apartments for the user along with a justification for your choice.
If there are no apartments that match the user's criteria, return "No apartments found".
"""


# In practice I want this to come from a conversation and more details with the user (should be an agent that talks with them about what they like, images of design they like, where they want to live, etc.)
user_profile_1 = """
I am looking for an apartment in the west village or chelsea,  with 1 bedroom, 1 bathrooms.
I love natural light, a nice view, and a short walk to the subway and citibike stations. 
I am willing to pay up to 4000 USD per month.
I do not want to live above a bar, but restaurants and cafes are great.
I want to be close to the 1,2,3 subway lines.
I must be within a 5 min walk of at least one the best coffee shops in the city.
"""


user_profile_2 = """
Looking for a studio or 1BR in Williamsburg or Greenpoint, Brooklyn.
Budget is $2500-3000/month. Must have laundry in building and allow cats.
I work from home so need good internet and a quiet space. 
Prefer top floor apartments with good light. No basement units.
Close to L or G train is essential. Want to be near McCarren Park.
Love the neighborhood vibe with vintage shops and indie cafes.
"""

user_profile_3 = """
Need a 2 bedroom apartment in Upper West Side or Morningside Heights.
Have a family with one child, so looking for $5000-6500/month.
Must be near good schools and playgrounds. 
Doorman building strongly preferred.
Need to be close to Central Park and 1/2/3 or B/C lines.
Want a family-friendly neighborhood with grocery stores nearby.
Elevator building is a must. No walk-ups above 2nd floor.
"""
user_profile_4 = """
Looking for luxury 1BR with home office space in Tribeca or Financial District.
Budget up to $7000/month. Must have doorman and gym in building.
Work in finance so need quick access to Wall Street area.
Want modern finishes, in-unit washer/dryer, and a balcony or terrace.
Building must have a package room and bike storage.
Prefer high floor with city views. 
Close to 4/5/6 or R/W lines. Whole Foods nearby is a must.
"""

# More tools:
# - analyze_image: analyze an image of an apartment and return a description of the apartment.


In [23]:
# OpenAI API Tools Configuration
tools = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search for information on the web by asking one or more questions. Returns answers to help with apartment hunting, neighborhood research, or general inquiries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "questions": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of questions to search for on the web. Each question should be clear and specific.",
                    }
                },
                "required": ["questions"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_apartment_dataset_schema",
            "description": "Get the schema of the apartment dataset and an example apartment listing.",
            "parameters": {
                "type": "object",
                "properties": {
                    "filepath": {
                        "type": "string",
                        "description": "Path to the JSON file containing apartment listings",
                    }
                },
                "required": ["filepath"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_apartment_values",
            "description": "Get all possible values for specified fields across all apartments in the dataset.",
            "parameters": {
                "type": "object",
                "properties": {
                    "filepath": {
                        "type": "string",
                        "description": "Path to the JSON file containing apartment listings",
                    },
                    "fields": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of field names to get unique values for (e.g., ['addr_hood', 'bedrooms', 'price'])",
                    },
                },
                "required": ["filepath", "fields"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "ask_user",
            "description": "Ask the user for more information about their apartment preferences or requirements.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask the user about their preferences, needs, or requirements",
                    }
                },
                "required": ["question"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_keywords_in_values",
            "description": "Search for apartments that contain any of the specified keywords in any of their field values.",
            "parameters": {
                "type": "object",
                "properties": {
                    "filepath": {
                        "type": "string",
                        "description": "Path to the JSON file containing apartment listings",
                    },
                    "keywords": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of keywords to search for across all apartment fields (case-insensitive)",
                    },
                },
                "required": ["filepath", "keywords"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_by_field",
            "description": "Search for apartments that contain keywords in specific fields only.",
            "parameters": {
                "type": "object",
                "properties": {
                    "filepath": {
                        "type": "string",
                        "description": "Path to the JSON file containing apartment listings",
                    },
                    "keywords": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of keywords to search for (case-insensitive)",
                    },
                    "fields": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of field names to search within (e.g., ['addr_hood', 'description', 'title'])",
                    },
                },
                "required": ["filepath", "keywords", "fields"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
]

In [ ]:
messages = [
    {"role": "system", "content": developer},
    {"role": "user", "content": user_profile_1},
]

turns = 0 # safety limmit
while turns < 30:
    print(f"Turn {turns}")
    completion = client.chat.completions.create(
        model=agent_model,
        messages=messages,
        tools=tools
    )
    messages.append(completion.choices[0].message)
    if completion.choices[0].message.tool_calls:
        async def run_all_tools():
            tasks = []
            for tool_call in completion.choices[0].message.tool_calls:
                name = tool_call.function.name
                args = json.loads(tool_call.function.arguments)
                print(f"Tool call: {name} with args: {args}\n")

                async def execute_single(tc=tool_call, n=name, a=args):
                    result = await call_function_async(n, a)
                    return {
                        "role": "tool",
                        "tool_call_id": tc.id,
                        "content": str(result)
                    }

                tasks.append(execute_single())

            # Run all tasks in parallel
            return await asyncio.gather(*tasks)

        # Execute all tool calls in parallel
        tool_results = asyncio.run(run_all_tools())
        messages.extend(tool_results)
        turns += len(tool_results)
    # If no tool calls, we stop and return the final answer
    else:
        break

print(messages[-1].content)

Turn 0
Tool call: search_by_field with args: {'filepath': '../data/streeteasy.json', 'keywords': ['West Village', 'Chelsea'], 'fields': ['addr_hood']}

Tool call: get_apartment_dataset_schema with args: {'filepath': '../data/streeteasy.json'}

Turn 2
Tool call: search_by_field with args: {'filepath': '../data/streeteasy.json', 'keywords': ['1'], 'fields': ['bedrooms']}

Turn 3
Tool call: search_by_field with args: {'filepath': '../data/streeteasy.json', 'keywords': ['West Village', 'Chelsea'], 'fields': ['addr_hood']}

Turn 4
Tool call: search_by_field with args: {'filepath': '../data/streeteasy.json', 'keywords': ['1'], 'fields': ['bedrooms']}

Turn 5
Tool call: search_by_field with args: {'filepath': '../data/streeteasy.json', 'keywords': ['West Village'], 'fields': ['addr_hood']}

Turn 6
Tool call: search_by_field with args: {'filepath': '../data/streeteasy.json', 'keywords': ['10 Gay Street'], 'fields': ['addr_street']}

Turn 7
Tool call: search_by_field with args: {'filepath': '..